In [1]:
!pip install scrapinghub

In [2]:
!pip install scrapinghub-autoextract

In [3]:
from autoextract.sync import request_raw

ModuleNotFoundError: No module named 'autoextract'

In [4]:
#!pip install scrapinghub-autoextract
#!pip install auto-extract

import pandas as pd
import numpy as np
import requests
import time
import os
#from autoextract.sync import request_raw

## If Only A Few URLs

In [ ]:


query = [{
    'url': 'http://example.com/article?id=24',
    'pageType': 'article'
}]
results = request_raw(query, api_key='[api key]')
print(results[0]['article'])

In [14]:
#import requests

response = requests.post(
    'https://autoextract.scrapinghub.com/v1/extract',
    auth=('531660b7c27448f1b355c16abd639291', ''),
    json=[{'url': 'https://www.msn.com/en-us/news/world/south-korea-has-reported-additional-48-cases-of-a-new-virus-raising-its-total-to-204/ar-BB10eeyv', 'pageType': 'article'}],
    timeout=605,
)

a = response.json()
print(a)

[{'query': {'id': '1620506668704-0d5a3d545efcfdbc', 'domain': 'msn.com', 'userAgent': 'python-requests/2.24.0', 'userQuery': {'url': 'https://www.msn.com/en-us/news/world/south-korea-has-reported-additional-48-cases-of-a-new-virus-raising-its-total-to-204/ar-BB10eeyv', 'pageType': 'article'}}, 'error': 'Downloader error: http404'}]


In [15]:
a[0].keys()

dict_keys(['query', 'error'])

## IF AT SCALE

In [ ]:
def ExtractNews(urls):
    Headline = []
    ArticleBody = []
    Authors = []
    DatePublished = []
    DateModified = []
    Domain = []
    NewsUrls = []
    ImageUrls = []
    VideoUrls = []
    AudioUrls = []
    Prop = []
    MissingIndexes = []
    
    a = 0
    
    for i in range(len(urls)):
        a += 1
        if a % 100 == 0:
            print(a)
        
        try:
            response = requests.post('https://autoextract.scrapinghub.com/v1/extract',
                                    auth=('531660b7c27448f1b355c16abd639291', ''),
                                    json=[{'url': urls[i], 'pageType': 'article'}],
                                    timeout=605
                                  )
            
            #pls check official document: https://doc.scrapinghub.com/autoextract/article.html
            #get headline
            try:
                Headline.append(response.json()[0]['article'].get('headline'))
            except:
                Headline.append('Error')
            #get articlebody     
            try:
                ArticleBody.append(response.json()[0]['article'].get('articleBody')) 
            except:
                ArticleBody.append("Error")
            #get authors
            try:
                Authors.append(response.json()[0]['article'].get('authorsList'))
            except:
                Authors.append('Error')
            #get published date
            try:
                DatePublished.append(response.json()[0]['article'].get('datePublished'))
            except:
                DatePublished.append('Error')
            #get modified date
            try:
                DateModified.append(response.json()[0]['article'].get('dateModified'))
            except:
                DateModified.append('Error')
            #get domain
            try:
                Domain.append(response.json()[0]['query'].get('domain'))
            except:
                Domain.append('Error')
            #get url
            try:
                NewsUrls.append(response.json()[0]['article'].get('url'))
            except:
                NewsUrls.append('Error')
            #get image url
            try:
                ImageUrls.append(response.json()[0]['article'].get('images'))
            except:
                ImageUrls.append('Error')  
            #get video url
            try:
                VideoUrls.append(response.json()[0]['article'].get('videoUrls'))
            except:
                VideoUrls.append('Error') 
            #get audio url
            try:
                AudioUrls.append(response.json()[0]['article'].get('audioUrls'))
            except:
                AudioUrls.append('Error') 
            #get Propobility of being a single page article
            try:
                Prop.append(response.json()[0]['article'].get('probability'))
            except:
                Prop.append('Error') 
            
        
        except: 
            MissingIndexes.append(urls[i])
            
        
            
        df = pd.DataFrame({
            'Head line' : Headline,
            'ArticleBody' : ArticleBody,
            'Authors' : Authors,
            'DatePublished' : DatePublished,
            'DateModified' : DateModified,
            'Domain' : Domain,
            'NewsUrls' : NewsUrls,
            'ImageUrls' : ImageUrls,
            'VideoUrls' : VideoUrls,
            'AudioUrls' : AudioUrls,
            'Prop' : Prop
        }
        )
             
    return df, MissingIndexes

In [ ]:
# run function
newfilename_list = []
path = '/Volumes/ZeDrive1/Projects/Covid_Bot Agenda setting/News Data/COVID-19_Media Cloud/Foxnews_2020/'
for root, dirs, files in os.walk(path):
    for filename in files:
        if "Foxnews" in filename and "._" not in filename:
            tic = time.time()
            data = pd.read_csv(path + filename)
            urls = list(data['url'])
            results = ExtractNews(urls)[0]
         
            newfilename = filename.replace('Foxnews','Foxnews_data')
            newfilename_list.append(newfilename)
            
            results.to_csv(path + newfilename)
            toc = time.time()
            print("Now is ", time.ctime(toc), ", we spent ", round(toc - tic, 2),  " for scraping ", filename, " data.")
            print("In total, there have ", str(len(data)), "URLs. We obtained ", str(len(results)), "news data.")
            
            
            